# CSCE 470 :: Information Storage and Retrieval :: Texas A&M University :: Fall 2018


# Homework 3 (and 4):  Recommenders

### 100 points [10% of your final grade; that's double!]

### Due: November 8, 2018

*Goals of this homework:* Put your knowledge of recommenders to work. 

*Submission Instructions (Google Classroom):* To submit your homework, rename this notebook as  `lastname_firstinitial_hw#.ipynb`. For example, my homework submission would be: `caverlee_j_hw3.ipynb`. Submit this notebook via **Google Classroom**. Your IPython notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so).

# Part 0: Movielens Data

For this first part, we're going to use part of the Movielens 100k dataset. Prior to the Netflix Prize, the Movielens data was **the** most important collection of movie ratings.

First off, we need to load the data (including u.user, u.item, and ua.base). Here, we provide you with some helper code to load the data using [Pandas](http://pandas.pydata.org/). Pandas is a nice package for Python data analytics.

You may need to install pandas doing something like:

`conda install --name cs470 pandas`

In [88]:
import subprocess
subprocess.call(['pip', 'install', 'pandas'])
subprocess.call(['pip', 'install', 'numpy'])
import pandas as pd
from statistics import stdev
import heapq
import math

# Load the user data
users_df = pd.read_csv('u.user', sep='|', names=['UserId', 'Age', 'Gender', 'Occupation', 'ZipCode'])

# Load the movies data: we will only use movie id and title for this homework
movies_df = pd.read_csv('u.item', sep='|', names=['MovieId', 'Title'], usecols=range(2), encoding = "ISO-8859-1")

# Load the ratings data: ignore the timestamps
ratings_df = pd.read_csv('ua.base', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))

# Working on three different data frames is a pain
# Let us create a single dataset by "joining" these three data frames
movie_ratings_df = pd.merge(movies_df, ratings_df)
movielens_df = pd.merge(movie_ratings_df, users_df)

movielens_df.head()

,MovieId,Title,UserId,Rating,Age,Gender,Occupation,ZipCode
0,1,Toy Story (1995),1,5,24,M,technician,85711
1,2,GoldenEye (1995),1,3,24,M,technician,85711
2,3,Four Rooms (1995),1,4,24,M,technician,85711
3,4,Get Shorty (1995),1,3,24,M,technician,85711
4,5,Copycat (1995),1,3,24,M,technician,85711


# Part 1. Let's find similar users [20 points]

Before we get to the actual task of building our recommender, let's familiarize ourselves with the Movielens data.

Pandas is really nice, since it let's us do simple aggregates. For example, we can find a specific user and take a look at that user's ratings. For example, for the user with user ID = 363, we have:

In [89]:
gbu = movielens_df.groupby('UserId')
gbm = movielens_df.groupby('MovieId')

# Information for a user
User363 = gbu.get_group(363)
User363[:1][["UserId", "Age", "Gender","Occupation", "ZipCode"]]

# Information for a movie
#Movie97 = gbm.get_group(97)
#Movie97[:1][["MovieId", "Title"]]

,UserId,Age,Gender,Occupation,ZipCode
23594,363,20,M,student,87501


In [90]:
# And then we can see his first 10 ratings:
User363[['Title', 'Rating']][:10]

,Title,Rating
23594,Toy Story (1995),2
23595,GoldenEye (1995),4
23596,Get Shorty (1995),5
23597,Copycat (1995),1
23598,Twelve Monkeys (1995),3
23599,Babe (1995),5
23600,Dead Man Walking (1995),3
23601,Seven (Se7en) (1995),5
23602,"Usual Suspects, The (1995)",5
23603,From Dusk Till Dawn (1996),4


Balderdash! Everyone agrees that Toy Story should be rated 5! Oh well, there's no accounting for taste.

Moving on, let's try our hand at finding similar users to this base user (user ID = 363). In each of the following, **find the top-10 most similar users** to this base user. You should use all of the user's ratings, not just the top-10 like we showed above. We're going to try different similarity methods and see what differences arise.

You should implement each of these similar methods yourself! 

###     Top-10 Most Similar Users Using

In [91]:
formulas = {
    #'jaccard': lambda x, y: w_diff(x, y)/w_union(x, y),
    #'cosine': lambda x, y: dot_prod(x, y)/(magn(x)*magn(y)),
    #'pearson': lambda x, y: covariance(x, y)/std_wrt(x, y),
    'default': lambda x, y: len(x.keys()&y.keys())/len(x.keys()|y.keys())
}
data_types = {
    'user': {
        'gb': gbu,
        'data': lambda id: gbu.get_group(id)[['MovieId','Rating']],
        'rats': lambda dat: dat.set_index('MovieId')['Rating'].to_dict()
    },
    'movie': {
        'gb': gbm,
        'data': lambda id: gbm.get_group(id)[['UserId','Rating']],
        'rats': lambda dat: dat.set_index('UserId')['Rating'].to_dict()
    }
}

# Fetch top-X most similar users/items; 'method' arg used to select approach
def most_similar(my_id, type='user', method='default', request=10):
    # Pull ratings for the input user/item
    my_data = data_types[type]['data'](my_id)
    my_rats = data_types[type]['rats'](my_data)

    heap = [] # size limited by max 'request' arg
    for ur_id,ur_data in data_types[type]['gb']:
        if ur_id == my_id: continue

        # Pull ratings for this user/item and compute score
        ur_rats = data_types[type]['rats'](ur_data)
        score = formulas[method](my_rats, ur_rats)
        # Store similarity score and userId in heap
        if len(heap) < request: heapq.heappush(heap, (score, ur_id))
        elif score > heap[0][0]: heapq.heappushpop(heap, (score, ur_id))
    heap.sort(reverse=True)
    return heap

####     Jaccard

In [92]:
# Set union and intersection, weighted by rating distance
def w_union(rat1, rat2):
    return len(rat1) + sum(0.2*abs(rat1.get(m, rat2[m]-5) - rat2[m]) for m in rat2)

def w_diff(rat1, rat2):
    return sum(1 - 0.2*abs(rat1.get(m, rat2[m]+5) - rat2[m]) for m in rat2)

formulas['jaccard'] = lambda x, y: w_diff(x, y)/w_union(x, y)

for u in most_similar(363, method='jaccard'):
    print("{:.9f} - user:{:03}".format(u[0], u[1]))

0.278445006 - user:276
0.262081784 - user:293
0.261770245 - user:435
0.256660746 - user:301
0.254002134 - user:429
0.253692762 - user:092
0.252059308 - user:916
0.251931994 - user:561
0.250000000 - user:268
0.249521623 - user:896


####     Cosine

In [93]:
def dot_prod(rat1, rat2):
    return sum(rat1.get(m, 0)*rat2[m] for m in rat2)

def magn(rat):
    return math.sqrt(sum(rat[m]**2 for m in rat))

formulas['cosine'] = lambda x, y: dot_prod(x, y)/(magn(x)*magn(y))

for u in most_similar(363, method='cosine'):
    print("{:.9f} - user:{:03}".format(u[0], u[1]))

0.603857486 - user:276
0.530818760 - user:864
0.529114652 - user:435
0.522761738 - user:303
0.522502505 - user:429
0.518805777 - user:896
0.512385644 - user:092
0.511879910 - user:682
0.510761576 - user:497
0.510252840 - user:222


#### Pearson

In [104]:
def std_wrt(rat1, rat2):
    if min(len(rat1), len(rat2)) < 2: return 1
    stdevs12 = stdev(rat1.values())*stdev(rat2.values())
    return 1 if stdevs12 == 0 else stdevs12

def covariance(rat1, rat2):
    m1 = sum(rat1.values())/len(rat1)
    m2 = sum(rat2.values())/len(rat2)
    return sum((rat1.get(m,m1) - m1)*(rat2[m] - m2) for m in rat2)/len(rat1)

formulas['pearson'] = lambda x, y: covariance(x, y)/std_wrt(x, y)

for u in most_similar(363, method='pearson'):
    print("{:.9f} - user:{:03}".format(u[0], u[1]))

0.269867397 - user:276
0.208027127 - user:889
0.204050740 - user:092
0.197552792 - user:293
0.190429526 - user:435
0.182419957 - user:758
0.179706735 - user:007
0.179420224 - user:268
0.176710693 - user:013
0.171415983 - user:643


### What are the differences among these three similarity methods? Which one do you prefer, why?

Jaccard is good at identifying users with similar watch lists, but the default implementation doesn't take actual rating scores into consideration, meaning despite being useful for finding users who have seen the same movies, those users may have very different dispositions when it comes to those movies. The Cosine similarity calculation effectively measures the difference in vector-angle between two users' ratings, meaning it will be a better indicator than Jaccard for finding users with similar preferences. However, Cosine is overly impacted by user bias or "shifts". What I mean by that is that not all users rate movies within the same range, some tend towards a higher mean rating while others tend towards a lower mean. Variations in different users' standard deviation of ratings can also impact Cosine similarity index to favor some users more than others. This is where Pearson similarity can improve the results by centering each user's ratings on their mean and dividing my their standard deviation.  This makes users with different rating biases much more comparable.<br>
My preferred method is Jaccard, primarily due to its simplicity and algorithmic efficiency. All of the methods used here run in easily under a second, but the standard implementations of Cosine and Pearson involve the use of a square root function, which is computationally costly.  Jaccard can also be adjusted to respect differences in ratings while still maintaining its superior time complexity, which is actually something I did in my implementation above.

# Part 2: User-User Collaborative Filtering: Similarity-Based Ratings Prediction [20 points]

Now let's estimate the rating of UserID 363 for the movie "Dances with Wolves (1990)" (MovieId 97) based on the similar users. Find the 10 nearest (most similiar by using Pearson) users who rated the movie "Dances with Wolves (1990)" and try different aggregate functions. Recall, there are many different ways to aggregate the ratings of the nearest neighbors. We'll try three popular methods:

In [95]:
# Helper function to get a user's rating for a given movie
def get_rat(uid, movie_id):
    data = gbu.get_group(uid)
    return list(data.loc[data['MovieId'] == movie_id, 'Rating'])

# Find 100 nearest users using Pearson
similar = most_similar(363, method='pearson', request=100)
# Trim to top 10 that have seen "Dances with Wolves" (MovieId=97)
sim_rat = [(s,u,get_rat(u, 97)[0]) for (s,u) in similar if get_rat(u, 97)][:10]
for u in sim_rat:
    print("{:.9f} - user:{:03} - r:{}".format(u[0], u[1], u[2]))

0.269867397 - user:276 - r:3
0.208027127 - user:889 - r:3
0.197552792 - user:293 - r:4
0.179706735 - user:007 - r:5
0.179420224 - user:268 - r:4
0.176710693 - user:013 - r:4
0.164794341 - user:001 - r:3
0.161367769 - user:429 - r:4
0.161149605 - user:880 - r:4
0.157990943 - user:561 - r:3


### Method 1: Average. 
The first is to simply average the ratings of the nearest neighbors:
$r_{c,s} = \frac{1}{N}\sum_{c'\in \hat{C}}r_{c',s}$

In [96]:
predicted = sum(r for (s,u,r) in sim_rat)/len(sim_rat)
print(predicted)

3.7


### Method 2: Weighted Average 1. 
The second is to take a weighted average, where we weight more "similar" neighbors higher: $r_{c,s} = k\sum_{c'\in \hat{C}}sim(c, c')\times r_{c',s}$

Choose a reasonable k so that r_{c,s} is between 1 to 5

In [97]:
# k = 1/sum(similarity of compared users)
predicted = sum(r*s for (s,u,r) in sim_rat)/sum(s for (s,u,r) in sim_rat)
print(predicted)

3.6655298870774486


### Method 3: Weighted Average 2. 
An alternative weighted average is to weight the differences between their ratings and their average rating (in essence to reward movies that are above the mean): $r_{c,s} = \bar{r}_c + k\sum_{c'\in \hat{C}}sim(c, c')\times (r_{c',s} - \bar{r}_{c'})$

Choose a reasonable k so that r_{c,s} is between 1 to 5

In [98]:
def get_mean(user_id):
    user_data = data_types['user']['data'](user_id)
    user_rats = data_types['user']['rats'](user_data)
    return sum(user_rats.values())/len(user_rats)
    
predicted = sum(s*(r - get_mean(u)) for (s,u,r) in sim_rat)/sum(s for (s,u,r) in sim_rat)
predicted += get_mean(363)
print(predicted)

3.375631881236712


# Part 3: Baseline Recommendation (Global) [20 points]

OK, so far we've built the basics of a user-user collaborative filtering approach; that is, we take a user, find similar users and then aggregate their ratings. 

An alternative approach is to consider just basic statistics of the movies and users themselves. This is the essence of the "baseline" recommender we discussed in class:

$b_{xi} = \mu + b_x + b_i$

where $b_{x,i}$ is the baseline estimate rating user x would give to item i, $\mu$ is the overall mean rating, $b_x$ is a user bias term, and $b_i$ is an item bias term.

For this part, let's once again estimate the rating of UserID 363 for the movie "Dances with Wolves (1990)" (MovieId 97), but this time using the baseline recommender.

In [112]:
# Mu (overall mean rating)
mu = sum(ratings_df['Rating'])/len(ratings_df['Rating'])
user_bias = {u:(sum(d['Rating'])/len(d['Rating'])) - mu for u,d in gbu}
item_bias = {m:(sum(d['Rating'])/len(d['Rating'])) - mu for m,d in gbm}

bxi = mu + user_bias[363] + item_bias[97]
print("Mean:", mu)
print("User362 bias:", user_bias[363])
print("Item97 bias:", item_bias[97])
print("Prediction:", bxi)

Mean: 3.5238268742409184
User362 bias: -0.47067072806151655
Item97 bias: 0.2515968545726408
Prediction: 3.3047530007520427


# Part 4: Local + Global Recommendation (Baseline + Item-Item CF) [20 points]

Our final recommender combines the global baseline recommender with an item-item local recommender. 

$\hat{r}_{xi} = b_{xi} + \dfrac{\sum_{j \in N(i;x)}s_{ij} \cdot (r_{xj} - b_{xj})}{\sum_{j \in N(i;x)}s_{ij}} $

where 
* $\hat{r}_{xi}$ is our estimated rating for what user x would give to item i.
* $s_{ij}$ is the similarity of items i and j.
* $r_{xj}$ is the rating of user x on item j.
* $N(i;x)$ is the set of items similar to item i that were rated by x.

You will need to make some design choices here about what similarity measure to use and what threshold to determine what items belong in $N(i;x)$.

Now show us what this estimates the rating of UserID 363 for the movie "Dances with Wolves (1990)" (MovieId 97) to be:

In [114]:
# Quick baseline recommendation
def get_bxi(u, i):
    return mu + user_bias[u] + item_bias[i]

In [115]:
N = 50
u = 363
m = 97

# Fetch similar movies using pearson correlation
similar = most_similar(m, 'movie', 'pearson', N)
sim_rat = [(s,i,get_rat(u, i)[0]) for (s,i) in similar if get_rat(u, i)]

rxi = bxi + sum(s*(r - get_bxi(u, i)) for (s,i,r) in sim_rat)/sum(s for (s,i,r) in sim_rat)
rxi = min(max(rxi, 1), 5)

print("Predicted:", rxi)

Predicted: 3.121941439082882


# Part 5. Putting it all together! [20 points]

Finally, we're going to experiment with our different methods to see which one performs the best on our special test set of "hidden" ratings. We have three big "kinds" of recommenders:

* User-User Collaborative Filtering
* Baseline Recommendation (Global)
* Local + Global Recommender


But within each, we have lots of design choices. For example, do we try Jaccard+Average or Jaccard+WeightedAverage1? Do we try Jaccard or Cosine or Pearson? What choice of k? Etc.

For this part, you should train your methods on a special train set (the base set, see below). Then report your results over the test set (see below). You should use RMSE as your metric of choice. Which method performs best? You will need to experiment with many different approaches, but ultimately, you should tell us the best 2 or 3 methods and report the RMSE you get.

In [ ]:
train = pd.read_csv('ua.base', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))
test = pd.read_csv('ua.test', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))


In [ ]:
# your code here 

*provide your best 2 or 3 methods, their RMSE, plus some discussion of why they did the best*

### BONUS: 
Can you do better? Find a way to improve the results!

In [ ]:
# your code here